Importing the necessary libraries for the analysis:

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series,DataFrame

Pointing the requests library to the content of the Wikipedia page and viewing the pages (commented out)

In [2]:
r=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
c=r.content
#c

In [3]:
soup=BeautifulSoup(c,"html.parser")
#print(soup.prettify())

Finding the table on the Wikipedia page, extracting information from the rows and excluding the header

In [4]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

In [5]:
#replacing the \n seen from viewing the table information in some cells

rows = table.find_all('tr')[1:]
rows[0].find_all('td')[2].text.replace("\n","")

'Not assigned'

In [6]:
#Viewing what the first cell looks like

rows[0]

<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>

In [7]:
#Creating 3 series for the 3 columns and appending extracted data to them

Postcode = []
Borough = []
Neighborhood = []


for item in rows:
    Postcode.append(item.find_all('td')[0].text.replace("\n",""))
    Borough.append(item.find_all('td')[1].text.replace("\n",""))
    Neighborhood.append(item.find_all('td')[2].text.replace("\n",""))
    

In [8]:
#Creating pandas series from the lists

Postcode = Series(Postcode)
Borough = Series(Borough)
Neighborhood = Series(Neighborhood)

In [9]:
#Concatenating all three series to a pandas DataFrame

df = pd.concat([Postcode,Borough,Neighborhood],axis=1)

In [10]:
#adding columns to DataFrame

df.columns = ['Postcode','Borough','Neighborhood']
#df

In [15]:
df.shape

(288, 3)

In [22]:
#Creating new DataFrame ignoring boroughs that are not assigned

df_new = df[df['Borough']!='Not assigned'].reset_index(drop=True)

In [23]:
df_new.shape

(211, 3)

In [63]:
#Viewing the df_new DataFrame (commented out)
#df_new

In [45]:
#Using groupby Postcode as well as applying lambda function to create new DataFrame

df_newer = df_new.groupby(['Postcode','Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()

In [58]:
#Viewing all neighborhoods designated as 'Not assigned'

df_newer[df_newer['Neighborhood']=='Not assigned']

,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [59]:
#Setting neighborhood that are 'Not assigned' to same value as Borough

df_newer.loc[df_newer['Neighborhood']=='Not assigned', ['Neighborhood']] = df_newer['Borough']

In [60]:
#Checking that it worked

df_newer[df_newer['Postcode']=='M7A']

,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [64]:
#Viewing the latest version of DataFrame

#df_newer

In [62]:
df_newer.shape

(103, 3)